In [1]:
import sys
sys.path.append('../src')
sys.path.append('../submissions')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from load import load_train_data, load_test_data
from evaluate import evaluate_model
from utils import save_submission, save_model, load_model
from encoding import freq_encode, get_house_volume
from datetime import datetime

pd.set_option('display.max_columns', None)

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, TargetEncoder, FunctionTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score

In [3]:
# Load training data
train_data = load_train_data(local=True)

In [4]:
# Prepare data for preprocessing and modelling
TARGET = 'damage_grade'

X = train_data.copy()
y = train_data.pop(TARGET)

In [5]:
# Split into train and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2,
                                                      stratify=y,
                                                      random_state=42)

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC


In [9]:

base_line_pipeline = load_model('../models/model_baseline.pickle')

y_train_ = y_train.copy()
y_valid_ = y_valid.copy()

new_model = XGBClassifier(random_state=42)

new_pipeline = base_line_pipeline.set_params(model=new_model)

# Evaluate model performance
if "xgboost" in str(type(new_model)):
    y_train_ = y_train_.apply(lambda x: int(x-1))
    y_valid_ = y_valid_.apply(lambda x: int(x-1))
    
score_valid, score_training = evaluate_model(new_pipeline, X_train, X_valid, y_train_, y_valid_)
print(f"{new_model}:")
print(f"F1-score (validation): {score_valid :.3f}")
print(f"F1-score (training): {score_training :.3f}")
print("________________________________________")

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...):
F1-score (validation): 0.745
F1-score (training): 0.745
________________________________________
